# Web Scraping with Beautiful Soup

* * * 

### Icons used in this notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
🥊 **Challenge**: Interactive exercise. We'll work through these in the workshop!<br>
⚠️ **Warning**: Heads-up about tricky stuff or common mistakes.<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
🎬 **Demo**: Showing off something more advanced – so you know what Python can be used for!<br>

### Learning Objectives
1. [Reflection: To Scape Or Not To Scrape](#when)
2. [Extracting and Parsing HTML](#extract)
3. [Scraping the Illinois General Assembly](#scrape)

<a id='when'></a>

# To Scrape Or Not To Scrape

When we'd like to access data from the web, we first have to make sure if the website we are interested in offers a Web API. Platforms like Twitter, Reddit, and the New York Times offer APIs. **Check out D-Lab's [Python Web APIs](https://github.com/dlab-berkeley/Python-Web-APIs) workshop if you want to learn how to use APIs.**

However, there are often cases when a Web API does not exist. In these cases, we may have to resort to web scraping, where we extract the underlying HTML from a web page, and directly obtain the information we want. There are several packages in Python we can use to accomplish these tasks. We'll focus two packages: Requests and Beautiful Soup.

Our case study will be scraping information on the [state senators of Illinois](http://www.ilga.gov/senate), as well as the [list of bills](http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True) each senator has sponsored. Before we get started, peruse these websites to take a look at their structure.

## Installation

We will use two main packages: [Requests](http://docs.python-requests.org/en/latest/user/quickstart/) and [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Go ahead and install these packages, if you haven't already:

INSTALACIÓN DE LIBRERÍAS 

In [1]:
#Instalación de la librería requests
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Instalación de la librería Beautifulsoup
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


We'll also install the `lxml` package, which helps support some of the parsing that Beautiful Soup performs:

In [ ]:
#Instalación del paquete lxml
%pip install lxml

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------  3.9/4.0 MB 24.1 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 21.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


IMPORTAR LIBRERÍAS

In [4]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

<a id='extract'></a>

# Extracting and Parsing HTML 

In order to succesfully scrape and analyse HTML, we'll be going through the following 4 steps:
1. Make a GET request
2. Parse the page with Beautiful Soup
3. Search for HTML elements
4. Get attributes and text of these elements

## Step 1: Make a GET Request to Obtain a Page's HTML

We can use the Requests library to:

1. Make a GET request to the page, and
2. Read in the webpage's HTML code.

The process of making a request and obtaining a result resembles that of the Web API workflow. Now, however, we're making a request directly to the website, and we're going to have to parse the HTML ourselves. This is in contrast to being provided data organized into a more straightforward `JSON` or `XML` output.

SE HARÁ UN REQUEST PARA OBTENER EL HMTL DE LA PÁGINA

In [5]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# View some output
print(src[:1000])

<!DOCTYPE html>
<html lang="en">
<head id="Head1">
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="content-type" content="text/html;charset=utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <meta charset="utf-8" />
    <meta charset="UTF-8">
    <!-- Meta Description -->
    <meta name="description" content="Welcome to the official government website of the Illinois General Assembly">
    <meta name="contactName" content="Legislative Information System">
    <meta name="contactOrganization" content="LIS Staff Services">
    <meta name="contactStreetAddress1" content="705 Stratton Office Building">
    <meta name="contactCity" content="Springfield">
    <meta name="contactZipcode" content="62706">
    <meta name="contactNetworkAddress" content="webmaster@ilga.gov">
    <meta name="contactPhoneNumber" content="217-782-3944">
    <meta name="contactFaxNumber" content="217-524-6059">
    <meta name


## Step 2: Parse the Page with Beautiful Soup

Now, we use the `BeautifulSoup` function to parse the reponse into an HTML tree. This returns an object (called a **soup object**) which contains all of the HTML in the original document.

If you run into an error about a parser library, make sure you've installed the `lxml` package to provide Beautiful Soup with the necessary parsing tools.

ANALIZAR LO DEL HTML Y DEVOLVER EL CONTENIDO DEL HTML EN UN OBJETO SOUP ORGANIZADO

In [6]:
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')
# Take a look
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="en">
 <head id="Head1">
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta charset="utf-8"/>
  <!-- Meta Description -->
  <meta content="Welcome to the official government website of the Illinois General Assembly" name="description"/>
  <meta content="Legislative Information System" name="contactName"/>
  <meta content="LIS Staff Services" name="contactOrganization"/>
  <meta content="705 Stratton Office Building" name="contactStreetAddress1"/>
  <meta content="Springfield" name="contactCity"/>
  <meta content="62706" name="contactZipcode"/>
  <meta content="webmaster@ilga.gov" name="contactNetworkAddress"/>
  <meta content="217-782-3944" name="contactPhoneNumber"/>
  <meta content="217-524-6059" name="contactFaxNumber"/>
  <meta content="State Of Illinois" name="originatorJur

The output looks pretty similar to the above, but now it's organized in a `soup` object which allows us to more easily traverse the page.

## Step 3: Search for HTML Elements

Beautiful Soup has a number of functions to find useful components on a page. Beautiful Soup lets you find elements by their:

1. HTML tags
2. HTML Attributes
3. CSS Selectors

Let's search first for **HTML tags**. 

The function `find_all` searches the `soup` tree to find all the elements with an a particular HTML tag, and returns all of those elements.

What does the example below do?

NOS PERMITE BUSCAR LOS ELEMENTOS HTML Y CSS QUE CONTIENE LA PÁGINA 

In [7]:
# Find all elements with a certain tag
a_tags = soup.find_all("a")
print(a_tags[:10])

[<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="af" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-za"></span> Afrikaans
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="sq" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-al"></span> Albanian
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="ar" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-ae"></span> Arabic
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="hy" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-am"></span> Armenian
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="az" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-az"></span> Azerbaijani
            

Because `find_all()` is the most popular method in the Beautiful Soup search API, you can use a shortcut for it. If you treat the BeautifulSoup object as though it were a function, then it’s the same as calling `find_all()` on that object. 

These two lines of code are equivalent:

ESTAS LÍNEAS SON EQUIVALENTES PORQUE soup.find_all() y soup() DEVUELVEN EL MISMO RESULTADO BUSCANDO LAS ETIQUETAS CON a

In [8]:
a_tags = soup.find_all("a")
a_tags_alt = soup("a")
print(a_tags[0])
print(a_tags_alt[0])

<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>
<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>


How many links did we obtain?

NOS DEVUELVE EL NÚMERO DE LINKS EN a_tags//270 LINKS

In [9]:
print(len(a_tags))

270


That's a lot! Many elements on a page will have the same HTML tag. For instance, if you search for everything with the `a` tag, you're likely to get more hits, many of which you might not want. Remember, the `a` tag defines a hyperlink, so you'll usually find many on any given page.

What if we wanted to search for HTML tags with certain attributes, such as particular CSS classes? 

We can do this by adding an additional argument to the `find_all`. In the example below, we are finding all the `a` tags, and then filtering those with `class_="sidemenu"`.

USANDO LA FUNCION soup() y AÑADIÉNDOLE EL FILTRO class_="sidemenu", NOS DEVEULVE UN SEGMENTO MÁS ESPECÍFICO DE DATOS

In [19]:
# Get only the 'a' tags in 'sidemenu' class
side_menus = soup("a", class_="sidemenu")
side_menus[:10]

[]

A more efficient way to search for elements on a website is via a **CSS selector**. For this we have to use a different method called `select()`. Just pass a string into the `.select()` to get all elements with that string as a valid CSS selector.

In the example above, we can use `"a.sidemenu"` as a CSS selector, which returns all `a` tags with class `sidemenu`.

REALIZA LOS MISMO QUE EL EJEMPLO ANTERIOR PERO SE UTILIZA CSS SELECTOR, Y SE UTILIZA EL MÉTODO select()

In [12]:
# Get elements with "a.sidemenu" CSS Selector.
selected = soup.select("a.sidemenu")
selected[:5]

[]

## 🥊 Challenge: Find All

Use BeautifulSoup to find all the `a` elements with class `mainmenu`.

SE USARÁ EL MÉTODO select() Y EL FILTRO ES "mainmenu"

In [13]:
# YOUR CODE HERE
selected = soup.select("a.mainmenu")
selected[:5]

[]

## Step 4: Get Attributes and Text of Elements

Once we identify elements, we want the access information in that element. Usually, this means two things:

1. Text
2. Attributes

Getting the text inside an element is easy. All we have to do is use the `text` member of a `tag` object:

CAMBIAMOS "sidemenu" POR "dropdown-item" PARA OBTENER UN RESULTADO. SE IMPRIMIRÁ LA POSICIÓN [0] DE LO QUE SE OBTIENE EN side_menu_links, FINALMENTE, IMPRIMIRÁ EL TIPO DE first_link

In [25]:
# Get all sidemenu links as a list
side_menu_links = soup.select("a.dropdown-item")
#print(len(side_menu_links))

# Examine the first link
first_link = side_menu_links[0]
print(first_link)

# What class is this variable?
print('Class: ', type(first_link))

<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>
Class:  <class 'bs4.element.Tag'>


It's a Beautiful Soup tag! This means it has a `text` member:

IMPRIME EL TEXTO QUE CONTIENE LA ETIQUETA

In [22]:
print(first_link.text)


 English
                            


Sometimes we want the value of certain attributes. This is particularly relevant for `a` tags, or links, where the `href` attribute tells us where the link goes.

💡 **Tip**: You can access a tag’s attributes by treating the tag like a dictionary:

IMPRIME LO QUE CONTIENE href

In [23]:
print(first_link['href'])

#


## 🥊 Challenge: Extract specific attributes

Extract all `href` attributes for each `mainmenu` URL.

SE OBTIENE LA LISTA DE LOS href CON EL MÉTODO get() DEL ARREGLO side_menu_links. 

In [ ]:
# YOUR CODE HERE

# Extraer todos los href
href_list = [link.get("href") for link in side_menu_links if link.get("href")]
#print(len(href_list))

# Mostrar los resultados
for href in href_list:
   print(href)

88


<a id='scrape'></a>

# Scraping the Illinois General Assembly

Believe it or not, those are really the fundamental tools you need to scrape a website. Once you spend more time familiarizing yourself with HTML and CSS, then it's simply a matter of understanding the structure of a particular website and intelligently applying the tools of Beautiful Soup and Python.

Let's apply these skills to scrape the [Illinois 98th General Assembly](http://www.ilga.gov/senate/default.asp?GA=98).

Specifically, our goal is to scrape information on each senator, including their name, district, and party.

## Scrape and Soup the Webpage

Let's scrape and parse the webpage, using the tools we learned in the previous section.

SE HACE EL REQUEST A AL URL, SE LEÉ EL CONTENIDO DE LA PÁGINA Y RETORNA LA INFORMACIÓN EN UN OBJETO SOUP

In [28]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")

## Search for the Table Elements

Our goal is to obtain the elements in the table on the webpage. Remember: rows are identified by the `tr` tag. Let's use `find_all` to obtain these elements.

NO SE ENCONTRO NINGUNA FILA CON TR, PERO CAMBIAMOS A DIV

In [30]:
# Get all table row elements
rows = soup.find_all("div")
len(rows)

543

⚠️ **Warning**: Keep in mind: `find_all` gets *all* the elements with the `tr` tag. We only want some of them. If we use the 'Inspect' function in Google Chrome and look carefully, then we can use some CSS selectors to get just the rows we're interested in. Specifically, we want the inner rows of the table:

IMPRIME EL CONTENIDO DE LAS PRIMERAS 5 POSICIONES DEL ARREGLO, USANDO CSS SELECTOR

In [33]:
# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('div div div')

for row in rows[:5]:
    print(row, '\n')

<div b-0yw6sxot5c="" class="modal-content">
<div b-0yw6sxot5c="" class="modal-header">
<h5 b-0yw6sxot5c="" class="modal-title" id="languageModalLabel">Select Language</h5>
<button aria-label="Close" b-0yw6sxot5c="" class="close" data-dismiss="modal" type="button">
<span aria-hidden="true" b-0yw6sxot5c="">×</span>
</button>
</div>
<div b-0yw6sxot5c="" class="modal-body">
<p b-0yw6sxot5c="">The Illinois General Assembly offers the Google Translate™ service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.</p>
<p b-0yw6sxot5c="">Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.</p>
<p b-0yw6sxot5c="">The English language version is always the official and authoritative version of this website.</p>
<p b-0yw6sxot5c="">NOTE: To return to the original English language version, select the "Show Original" button on the Google Translate™ menu bar at the top o

It looks like we want everything after the first two rows. Let's work with a single row to start, and build our loop from there.

IMPRIME LA POSICION 2 DEL ARREGLO rows 

In [34]:
example_row = rows[2]
print(example_row.prettify())

<div b-0yw6sxot5c="" class="modal-body">
 <p b-0yw6sxot5c="">
  The Illinois General Assembly offers the Google Translate™ service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.
 </p>
 <p b-0yw6sxot5c="">
  Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.
 </p>
 <p b-0yw6sxot5c="">
  The English language version is always the official and authoritative version of this website.
 </p>
 <p b-0yw6sxot5c="">
  NOTE: To return to the original English language version, select the "Show Original" button on the Google Translate™ menu bar at the top of the window.
 </p>
 <div b-0yw6sxot5c="" class="dropdownx">
  <button b-0yw6sxot5c="" class="btn btn-secondary dropdown-toggle" data-toggle="dropdown" id="languageDropdownMenuButton" type="button">
   <i b-0yw6sxot5c="" class="fas fa-globe">
   </i>
   Choose Language
  </button>
  <div aria-labelledby="lang

Let's break this row down into its component cells/columns using the `select` method with CSS selectors. Looking closely at the HTML, there are a couple of ways we could do this.

* We could identify the cells by their tag `td`.
* We could use the the class name `.detail`.
* We could combine both and use the selector `td.detail`.

IMPRIMIR CONTENIDO DE LA ETIQUETA p

In [36]:
for cell in example_row.select('p'):
    print(cell)
print()

for cell in example_row.select('.detail'):
    print(cell)
print()

for cell in example_row.select('p.detail'):
   print(cell)
print()

<p b-0yw6sxot5c="">The Illinois General Assembly offers the Google Translate™ service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.</p>
<p b-0yw6sxot5c="">Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.</p>
<p b-0yw6sxot5c="">The English language version is always the official and authoritative version of this website.</p>
<p b-0yw6sxot5c="">NOTE: To return to the original English language version, select the "Show Original" button on the Google Translate™ menu bar at the top of the window.</p>





We can confirm that these are all the same.

CONFIRMAR QUE SE CUMPLE O NO

In [43]:
assert example_row.select('td') == example_row.select('.detail') == example_row.select('td.detail')

Let's use the selector `td.detail` to be as specific as possible.

SE GUARDA EN detail_cells LA INFORMACIÓN ESPECÍFICA DE ESA FILA

In [ ]:
# Select only those 'td' tags with class 'detail' 
detail_cells = example_row.select('p.detail')
detail_cells

[]

Most of the time, we're interested in the actual **text** of a website, not its tags. Recall that to get the text of an HTML element, we use the `text` member:

IMPRIME EL CONTENIDO QUE TIENE LA ETIQUETA QUE SE VE EN LA WEBSITE

In [46]:
# Keep only the text in each of those cells
row_data = [cell.text for cell in detail_cells]

print(row_data)

[]


Looks good! Now we just use our basic Python knowledge to get the elements of this list that we want. Remember, we want the senator's name, their district, and their party.

IMPRIME LOS NOMBRES, DISTRITO Y EL PARTIDO POLÍTICO AL QUE PERTENECEN, EL CUAL SE MUESTRA EN LA PÁGINA WEB

In [47]:
print(row_data[0]) # Name
print(row_data[3]) # District
print(row_data[4]) # Party

IndexError: list index out of range

## Getting Rid of Junk Rows

We saw at the beginning that not all of the rows we got actually correspond to a senator. We'll need to do some cleaning before we can proceed forward. Take a look at some examples:

IMPRIME EL CONTENIDO DEL ARREGLO ROW CON SALTOS DE LÍNEA, CON EL FIN DE LIMPIAR LA INFORMACIÓN ANTES DE PROCESARLA

In [48]:
print('Row 0:\n', rows[0], '\n')
print('Row 1:\n', rows[1], '\n')
print('Last Row:\n', rows[-1])

Row 0:
 <div b-0yw6sxot5c="" class="modal-content">
<div b-0yw6sxot5c="" class="modal-header">
<h5 b-0yw6sxot5c="" class="modal-title" id="languageModalLabel">Select Language</h5>
<button aria-label="Close" b-0yw6sxot5c="" class="close" data-dismiss="modal" type="button">
<span aria-hidden="true" b-0yw6sxot5c="">×</span>
</button>
</div>
<div b-0yw6sxot5c="" class="modal-body">
<p b-0yw6sxot5c="">The Illinois General Assembly offers the Google Translate™ service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.</p>
<p b-0yw6sxot5c="">Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.</p>
<p b-0yw6sxot5c="">The English language version is always the official and authoritative version of this website.</p>
<p b-0yw6sxot5c="">NOTE: To return to the original English language version, select the "Show Original" button on the Google Translate™ menu bar at t

When we write our for loop, we only want it to apply to the relevant rows. So we'll need to filter out the irrelevant rows. The way to do this is to compare some of these to the rows we do want, see how they differ, and then formulate that in a conditional.

As you can imagine, there a lot of possible ways to do this, and it'll depend on the website. We'll show some here to give you an idea of how to do this.

HAY MUCHAS MANERAS DE ESCOGER LAS FILAS RELEVANTES, UNA DE ELLAS ES OBTENIENDO LA LONGITUD DE LAS POSICIONES DEL ARREGLO

In [49]:
# Bad rows
print(len(rows[0]))
print(len(rows[1]))

# Good rows
print(len(rows[2]))
print(len(rows[3]))

7
5
11
7


Perhaps good rows have a length of 5. Let's check:

ESTA ES OTRA ALTERNATIVA DE VER SI SON BUENAS FILAS, EN ESTE CASO SU LONGITUD ES IGUAL A 5

In [50]:
good_rows = [row for row in rows if len(row) == 5]

# Let's check some rows
print(good_rows[0], '\n')
print(good_rows[-2], '\n')
print(good_rows[-1])

<div b-0yw6sxot5c="" class="modal-header">
<h5 b-0yw6sxot5c="" class="modal-title" id="languageModalLabel">Select Language</h5>
<button aria-label="Close" b-0yw6sxot5c="" class="close" data-dismiss="modal" type="button">
<span aria-hidden="true" b-0yw6sxot5c="">×</span>
</button>
</div> 

<div class="row pt-4">
<div class="col-sm-1"></div>
<div class="col-sm-11">
<button class="float-left btn btn-secondary m-1" id="about-cookies-btn" type="button"><span aria-hidden="true">Read About Cookies</span></button>
<button aria-label="Close" class="float-left btn btn-secondary m-1" data-cookie-string=".AspNet.Consent=yes; expires=Sun, 26 Jul 2026 16:56:37 GMT; path=/; secure; samesite=strict" data-dismiss="alert" type="button"><span aria-hidden="true">Accept &amp; Continue</span></button>
</div>
</div> 

<div class="col-sm-11">
<button class="float-left btn btn-secondary m-1" id="about-cookies-btn" type="button"><span aria-hidden="true">Read About Cookies</span></button>
<button aria-label="Clo

We found a footer row in our list that we'd like to avoid. Let's try something else:

EVITAR LA ÚLTIMA FILA

In [51]:
rows[2].select('td.detail') 

[]

SE CLASIFICA ENTRE FILAS BUENAS Y MALAS. DE LAS FILAS BUENAS SE SELECCIONA EL DETALLE E IMPRIME TANTO FILAS BUENAS CÓMO MALAS

In [52]:
# Bad row
print(rows[-1].select('td.detail'), '\n')

# Good row
print(rows[5].select('td.detail'), '\n')

# How about this?
good_rows = [row for row in rows if row.select('td.detail')]

print("Checking rows...\n")
print(good_rows[0], '\n')
print(good_rows[-1])

[] 

[] 

Checking rows...



IndexError: list index out of range

Looks like we found something that worked!

## Loop it All Together

Now that we've seen how to get the data we want from one row, as well as filter out the rows we don't want, let's put it all together into a loop.

SE DEFINE UNA LISTA EN DONDE SE ALMACENARÁN LAS FILAS QUE SON VÁLIDAS, LUEGO SE HACE UN BUCLE EN DONDE SE OBTIENE EL DETALLE Y DE ESTAS SE SACA EL TEXTO QUE SE VE REFLEJADO EN LA PÁGINA WEB, SE ALMACENA EN VARIABLES Y SE GUARDA EN LA LISTA

In [53]:
# Define storage list
members = []

# Get rid of junk rows
valid_rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in valid_rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    # Store in a tuple
    senator = (name, district, party)
    # Append to list
    members.append(senator)

IMPRIME LA LONGITUD DE LA LISTA MEMBERS

In [54]:
# Should be 61
len(members)

0

Let's take a look at what we have in `members`.

IMPRIME EL CONTENIDO DE LA LISTA MIEMBROS

In [55]:
print(members[:5])

[]


## 🥊  Challenge: Get `href` elements pointing to members' bills 

The code above retrieves information on:  

- the senator's name,
- their district number,
- and their party.

We now want to retrieve the URL for each senator's list of bills. Each URL will follow a specific format. 

The format for the list of bills for a given senator is:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

to get something like:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

in which `MEMBER_ID=1911`. 

You should be able to see that, unfortunately, `MEMBER_ID` is not currently something pulled out in our scraping code.

Your initial task is to modify the code above so that we also **retrieve the full URL which points to the corresponding page of primary-sponsored bills**, for each member, and return it along with their name, district, and party.

Tips: 

* To do this, you will want to get the appropriate anchor element (`<a>`) in each legislator's row of the table. You can again use the `.select()` method on the `row` object in the loop to do this — similar to the command that finds all of the `td.detail` cells in the row. Remember that we only want the link to the legislator's bills, not the committees or the legislator's profile page.
* The anchor elements' HTML will look like `<a href="/senate/Senator.asp/...">Bills</a>`. The string in the `href` attribute contains the **relative** link we are after. You can access an attribute of a BeatifulSoup `Tag` object the same way you access a Python dictionary: `anchor['attributeName']`. See the <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> for more details.
* There are a _lot_ of different ways to use BeautifulSoup to get things done. whatever you need to do to pull the `href` out is fine.

The code has been partially filled out for you. Fill it in where it says `#YOUR CODE HERE`. Save the path into an object called `full_path`.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Create empty list to store our data
members = []

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
# Get rid of junk rows
rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail') 
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]

    # YOUR CODE HERE
    full_path = ''

    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

In [ ]:
# Uncomment to test 
# members[:5]

## 🥊  Challenge: Modularize Your Code

Turn the code above into a function that accepts a URL, scrapes the URL for its senators, and returns a list of tuples containing information about each senator. 

In [ ]:
# YOUR CODE HERE
def get_members(url):
    return [___]


In [ ]:
# Test your code
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = get_members(url)
len(senate_members)

## 🥊 Take-home Challenge: Writing a Scraper Function

We want to scrape the webpages corresponding to bills sponsored by each bills.

Write a function called `get_bills(url)` to parse a given bills URL. This will involve:

  - requesting the URL using the <a href="http://docs.python-requests.org/en/latest/">`requests`</a> library
  - using the features of the `BeautifulSoup` library to find all of the `<td>` elements with the class `billlist`
  - return a _list_ of tuples, each with:
      - description (2nd column)
      - chamber (S or H) (3rd column)
      - the last action (4th column)
      - the last action date (5th column)
      
This function has been partially completed. Fill in the rest.

In [ ]:
def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src)
    rows = soup.select('tr')
    bills = []
    for row in rows:
        # YOUR CODE HERE
        bill_id =
        description =
        chamber =
        last_action =
        last_action_date =
        bill = (bill_id, description, chamber, last_action, last_action_date)
        bills.append(bill)
    return bills

In [ ]:
# Uncomment to test your code
# test_url = senate_members[0][3]
# get_bills(test_url)[0:5]

### Scrape All Bills

Finally, create a dictionary `bills_dict` which maps a district number (the key) onto a list of bills (the value) coming from that district. You can do this by looping over all of the senate members in `members_dict` and calling `get_bills()` for each of their associated bill URLs.

**NOTE:** please call the function `time.sleep(1)` for each iteration of the loop, so that we don't destroy the state's web site.

In [ ]:
# YOUR CODE HERE


In [ ]:
# Uncomment to test your code
# bills_dict[52]